Let's start importing all the libraries that we need:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from IPython.display import clear_output

How our data looks like?
1. Read the data using pandas
1. See the shape, the information that we have and the images
1. Plot how many images we have for every class, is it balanced?


In [ ]:
data = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")

In [ ]:
print("Information in train.csv file: ", list(data.columns))
print("We have " + str(data.shape[0]) + " images for training process with " + str(data.shape[1]-1) + " classes" )

In [ ]:
print(data.head())

We don't really need the "image_id" column, so let's drop it

In [ ]:
data.pop("image_id")
print(data.shape)

Let's see how many images we have for each class

In [ ]:
idx = {}
c = {1:"healty", 2: "multiple", 3:"rust", 4:"scab"}
d = {1:0, 2:0, 3:0, 4:0}
labels =  np.array((data))
for i in range(labels.shape[0]):
    for j in range(1,labels.shape[1]+1):
        if labels[i,j-1] == 1:
            d[j] += 1
            if j not in idx:
                idx[j] = i

for key, value in d.items():
    plt.bar(c[key], value)
plt.ylabel("Quantity")
plt.xlabel("Classes")
plt.show()

As you can see, we don't have many images with multiple diseases, therefore, our cnn may struggle with this specific class. Now let's see how the images look like:

In [ ]:
c = ["Healthy", "Multiple Diseases", "Rust", "Scab"]
directory = "../input/plant-pathology-2020-fgvc7/images"
healthy = plt.imread(os.path.join(directory, "Train_"+str(idx[1])+".jpg"))
Multiple = plt.imread(os.path.join(directory, "Train_"+str(idx[2])+".jpg"))
Rust = plt.imread(os.path.join(directory, "Train_"+str(idx[3])+".jpg"))
Scab = plt.imread(os.path.join(directory, "Train_"+str(idx[4])+".jpg"))
imgs = np.array(([healthy, Multiple, Rust, Scab]))

fig=plt.figure(figsize=(8, 8))
rows = 2
columns = 2
for i in range(1, 5):
    fig.add_subplot(rows, columns, i)
    plt.title(c[-i])
    plt.imshow(imgs[-i,:,:,:])
plt.show()

Ok, we've seen how our data looks like, now define a function that get a batch for the training and at the same time normalize them. 

In [ ]:
def get_batch(n,size, healthy, directory, number, k = "Train"):
    # Used to get the batch for the training or testing dataset
    # Arguments:
    # n(int): ID of the image you will use from the dataset
    # size(int): Batch size
    # healthy(numpy.ndarray): Image taken from the dataset useful to know the shape of the dataset
    # directory(string): Directory where you are reading the images
    # number(int): number of images in your dataset
    # k(string): Specify if you are taking the images from the Train dataset or the Test dataset
    
    if k == "Train":
        if n + size > number:
            imgs = np.zeros((number-n,healthy.shape[0], healthy.shape[1], healthy.shape[2]))
            for h, i in enumerate(range(n,number)):
                imgs[h,:,:,:] = plt.imread(directory+"/Train_"+str(i)+".jpg")
        else:
            imgs = np.zeros((size,healthy.shape[0], healthy.shape[1], healthy.shape[2]))
            for h,i in enumerate(range(n,n+size)):
                img = plt.imread(directory+"/Train_"+str(i)+".jpg")
                if img.shape == healthy.shape:
                  imgs[h,:,:,:] = img
                else:
                  img = np.rot90(img)
                  imgs[h,:,:,:] = img
        imgs /= 255.0
    elif k == "Test":
        if n + size > number:
            imgs = np.zeros((number-n,healthy.shape[0], healthy.shape[1], healthy.shape[2]))
            for h, i in enumerate(range(n,number)):
                imgs[h,:,:,:] = plt.imread(directory+"/Test_"+str(i)+".jpg")
        else:
            for h,i in enumerate(range(n,n+size)):
                imgs = np.zeros((size,healthy.shape[0], healthy.shape[1], healthy.shape[2]))
                imgs[h,:,:,:] = plt.imread(directory+"/Test_"+str(i)+".jpg")
        imgs /= 255.0
    else:
        print("Directory of Training or Testing images not well established")
    return imgs

Because the images are too big, we are going to chunk the images making N patches of all the images and get our batch for the training. So we are going set random points in the images (set it as our central point for a single patch) and store it in a list. This will help us to reduce the amount of ram that we use.

In [ ]:
def get_patches(healthy, directory, batch_size = 10, number_images = 100, 
                number_patches = 5, size_patches = 128, k = "Train", ID= 1):
    # Used to get the batch for the training or testing dataset
    # Arguments:
    # healthy(numpy.ndarray): Image taken from the dataset useful to know the shape of the dataset
    # directory(string): Directory where you are reading the images
    # batch_size(int): batch size
    # number_images(int): number of images in your dataset
    # number_patches(int): number of patches you will get for each readed image
    # size_patches(int): size of the patch
    # k(string): Specify if you are taking the images from the Train dataset or the Test dataset
    imgs = get_batch(ID, batch_size, healthy, directory, number_images, k)
    w = healthy.shape[1]
    h = healthy.shape[0]
    patches = np.zeros((imgs.shape[0]*number_patches, size_patches, size_patches, 3))
    l = [] #List with our central points
    x = w
    y = h
    while(len(l) < number_patches*2):
        x = np.random.randint(0,w) #random point in the x axis
        y = np.random.randint(0,h) #random point in the y axis
        #We need to make sure that the size of the patches doesn't get out of the images shape
        if w >= x+(size_patches/2) and x-(size_patches/2) >= 0 and h >= y+(size_patches/2) and y-(size_patches/2) >= 0:
            l.append(x)
            l.append(y)
            
    for i in range(0,imgs.shape[0]):
        for k, j in enumerate(range(0,number_patches*2, 2)):
            patches[k+(i*number_patches),:,:,:] = imgs[i,l[j+1]-int(size_patches/2):l[j+1]+int(size_patches/2), l[j]-int(size_patches/2):l[j]+int(size_patches/2),:]
    return patches

#The function give us N patches of the image that we get in the get_batch function
batch_size = 1
patches = get_patches(healthy, directory, batch_size, labels.shape[0], 6, 600, ID=0)
print(patches.shape)
#Let's visualize some patches
fig=plt.figure(figsize=(8, 8))
rows = 2
columns = 3
for i in range(1, (rows*columns)+1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(patches[-i, :, :, :])
plt.show()

Also, let's define a function to evaluate our model.

In [ ]:
def accuracy(batch_size, model, labels, healthy, directory, ID=1, verbose = False):
    # Used to measure the accuracy of the model
    # Arguments:
    # batch_size(int): batch_size
    # model(tf model class): convolutional neural network model
    # labels(int): labels used for the training
    # healthy(numpy.ndarray): Image taken from the dataset useful to know the shape of the dataset
    # directory(string): Directory where you are reading the images
    n_patches = 5 #Use odd numbers to get a favor democracy for each label
    patches = get_patches(healthy, directory, batch_size = batch_size, number_images = labels.shape[0], 
                number_patches = n_patches, size_patches = 256, k = "Train", ID=ID)

    n = np.random.randint(0,labels.shape[0]-batch_size)

    l = np.zeros((batch_size, 1))


    for i in range(ID, batch_size):
      l[i] = tf.argmax(labels[i,:])

    #print(labels[ID:ID+batch_size, :])
    predictions = model(patches)
    acc = 0
    Result = np.zeros((n_patches))
    for i in range(0, batch_size):
        for k in range(n_patches):
            Result[k] = int(tf.argmax(predictions[(i*n_patches)+k]))

        counts = np.bincount(Result.astype("int32"))
        R = np.argmax(counts)

        if verbose == True:
          print("Predicted value: " + str(R) + " Real value: " + str(l[i]))
        
        if R == l[i]:
            acc += 1
    acc /= batch_size
    return acc

It's time to define our cnn:

In [ ]:
def build_cnn():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(24,(3,3), activation="relu"))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Conv2D(64,(3,3), activation="relu"))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Conv2D(128,(3,3), activation="relu"))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    return model
    

Ok, let's get our model instance and prepare the optimizer

In [ ]:
model = build_cnn()

In [ ]:
optimizer = tf.keras.optimizers.Adam()

Training time!!!! Let's use a batch size of 128 and 100 epochs

In [ ]:
batch_size = 25
acc_batch = 7
epochs = 2
n_patches = 10
history = []
x = list(range(100+int(epochs*(labels.shape[0]/batch_size))))

for i in range(epochs):
    for j in range(0, labels.shape[0], batch_size):
        #images_b = get_batch(j, batch_size, healthy, directory, labels.shape[0])
        images_b = get_patches(healthy, directory, batch_size, labels.shape[0], n_patches, 256, ID=j)
        if j+batch_size > labels.shape[0]:
          labels_b = labels[j:]
          l = np.zeros(((labels_b.shape[0]*n_patches,1))) #We have to put a label on each patch and since is the same label for the next n_patches then we just make it bigger
        else:
          labels_b = labels[j:j+batch_size]
          l = np.zeros(((batch_size*n_patches,1)))
        for e in range(labels_b.shape[0]):
            l[e*(n_patches):(e*(n_patches))+n_patches] = tf.argmax(labels_b[e,:])
        with tf.GradientTape() as tape:
            predictions = model(images_b)
            loss = tf.keras.losses.sparse_categorical_crossentropy(l, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        ID_T = np.random.randint(0,1820-acc_batch)
        history.append(accuracy(batch_size = acc_batch, model = model, labels = labels, healthy = healthy, directory = directory, ID = ID_T, verbose = False))
        plt.plot(x[:len(history)], history)
        plt.xlabel("step")
        plt.ylabel("accuracy")
        plt.show()
        clear_output(wait=True)
